In [82]:
from src.Dataset import Dataset
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd

In [83]:
# train=Dataset(filename="fr_gsd-ud-train.conllu")
# train=Dataset(filename="fi_tdt-ud-train.conllu")
train=Dataset(filename="en_ewt-ud-train.conllu")

In [84]:
#3
# X = [{"word": train.types[train.X[i]],
#       "left_neighbor":train.types[train.X[i-1]],
#       "right_neighbor":train.types[train.X[(i+1) % train.nb_tokens]],
#       "pos_in_sentence":0,
#       "after_comma":0,
#       "before_comma":0
#       } for i in range(train.nb_tokens)]

# for i in range(train.nb_sentences):
#     X[train.start_sentences[i]]["left_neighbor"]=""
#     X[train.start_sentences[i]+train.len_sentences[i]-1]["right_neighbor"]=""
#     for j in range(train.len_sentences[i]):
#         s=train.start_sentences[i]
#         if train.len_sentences[i]>1:
#             X[s+j]["pos_in_sentence"]=(j/(train.len_sentences[i]-1))
#         X[s+j]["after_comma"]=(X[s+j]["left_neighbor"]==",")
#         X[s+j]["before_comma"]=(X[s+j]["right_neighbor"]==",")

#4
X = [{"word": train.types[train.X[i]],
      "left_neighbor":train.types[train.X[i-1]],
      "right_neighbor":train.types[train.X[(i+1) % train.nb_tokens]],
      "pos_in_sentence":0,
      "end_word3":train.types[train.X[i]][-3:],
      "end_word2":train.types[train.X[i]][-2:],
      } for i in range(train.nb_tokens)]

for i in range(train.nb_sentences):
    X[train.start_sentences[i]]["left_neighbor"]=""
    X[train.start_sentences[i]+train.len_sentences[i]-1]["right_neighbor"]=""
    for j in range(train.len_sentences[i]):
        s=train.start_sentences[i]
        if train.len_sentences[i]>1:
            X[s+j]["pos_in_sentence"]=(j/(train.len_sentences[i]-1))


#5
# X = [{"word": train.types[train.X[i]],
#       "left_neighbor":train.types[train.X[i-1]],
#       "right_neighbor":train.types[train.X[(i+1) % train.nb_tokens]],
#       "pos_in_sentence":0,
#       "end_word3":train.types[train.X[i]][-3:],
#       "end_word2":train.types[train.X[i]][-2:],
#       "left_pos":""
#       } for i in range(train.nb_tokens)]

# for i in range(train.nb_sentences):
#     X[train.start_sentences[i]]["left_neighbor"]=""
#     X[train.start_sentences[i]+train.len_sentences[i]-1]["right_neighbor"]=""
#     for j in range(train.len_sentences[i]):
#         s=train.start_sentences[i]
#         if train.len_sentences[i]>1:
#             X[s+j]["pos_in_sentence"]=(j/(train.len_sentences[i]-1))
#         X[s+j]["left_end_word3"]=X[s+j]["left_neighbor"][-3:]
#         X[s+j]["left_end_word2"]=X[s+j]["left_neighbor"][-2:]
#         X[s+j]["right_end_word3"]=X[s+j]["right_neighbor"][-3:]
#         X[s+j]["right_end_word2"]=X[s+j]["right_neighbor"][-2:]
#         if X[s+j]["left_neighbor"]!="":
#             X[s+j]["left_pos"]=train.pos[train.y[s+j-1]]

#6
# X = [{"word": train.types[train.X[i]]} for i in range(train.nb_tokens)]

#7
X = [{"word": train.types[train.X[i]],
      "left_neighbor":train.types[train.X[i-1]],
      "right_neighbor":train.types[train.X[(i+1) % train.nb_tokens]],
      "pos_in_sentence":0,
      "end_word3":train.types[train.X[i]][-3:],
      "end_word2":train.types[train.X[i]][-2:],
      "left_pos":""
      } for i in range(train.nb_tokens)]

for i in range(train.nb_sentences):
    X[train.start_sentences[i]]["left_neighbor"]=""
    X[train.start_sentences[i]+train.len_sentences[i]-1]["right_neighbor"]=""
    for j in range(train.len_sentences[i]):
        s=train.start_sentences[i]
        if train.len_sentences[i]>1:
            X[s+j]["pos_in_sentence"]=(j/(train.len_sentences[i]-1))
        X[s+j]["left_end_word3"]=X[s+j]["left_neighbor"][-3:]
        X[s+j]["left_end_word2"]=X[s+j]["left_neighbor"][-2:]
        X[s+j]["right_end_word3"]=X[s+j]["right_neighbor"][-3:]
        X[s+j]["right_end_word2"]=X[s+j]["right_neighbor"][-2:]
        if X[s+j]["left_neighbor"]!="":
            X[s+j]["left_pos"]=train.pos[train.y[s+j-1]]

In [85]:
X

[{'word': 'Al',
  'left_neighbor': '',
  'right_neighbor': '-',
  'pos_in_sentence': 0.0,
  'end_word3': 'Al',
  'end_word2': 'Al',
  'left_pos': '',
  'left_end_word3': '',
  'left_end_word2': '',
  'right_end_word3': '-',
  'right_end_word2': '-'},
 {'word': '-',
  'left_neighbor': 'Al',
  'right_neighbor': 'Zaman',
  'pos_in_sentence': 0.03571428571428571,
  'end_word3': '-',
  'end_word2': '-',
  'left_pos': 'PROPN',
  'left_end_word3': 'Al',
  'left_end_word2': 'Al',
  'right_end_word3': 'man',
  'right_end_word2': 'an'},
 {'word': 'Zaman',
  'left_neighbor': '-',
  'right_neighbor': ':',
  'pos_in_sentence': 0.07142857142857142,
  'end_word3': 'man',
  'end_word2': 'an',
  'left_pos': 'PUNCT',
  'left_end_word3': '-',
  'left_end_word2': '-',
  'right_end_word3': ':',
  'right_end_word2': ':'},
 {'word': ':',
  'left_neighbor': 'Zaman',
  'right_neighbor': 'American',
  'pos_in_sentence': 0.10714285714285714,
  'end_word3': ':',
  'end_word2': ':',
  'left_pos': 'PROPN',
  'left_

In [86]:
X

[{'word': 'Al',
  'left_neighbor': '',
  'right_neighbor': '-',
  'pos_in_sentence': 0.0,
  'end_word3': 'Al',
  'end_word2': 'Al',
  'left_pos': '',
  'left_end_word3': '',
  'left_end_word2': '',
  'right_end_word3': '-',
  'right_end_word2': '-'},
 {'word': '-',
  'left_neighbor': 'Al',
  'right_neighbor': 'Zaman',
  'pos_in_sentence': 0.03571428571428571,
  'end_word3': '-',
  'end_word2': '-',
  'left_pos': 'PROPN',
  'left_end_word3': 'Al',
  'left_end_word2': 'Al',
  'right_end_word3': 'man',
  'right_end_word2': 'an'},
 {'word': 'Zaman',
  'left_neighbor': '-',
  'right_neighbor': ':',
  'pos_in_sentence': 0.07142857142857142,
  'end_word3': 'man',
  'end_word2': 'an',
  'left_pos': 'PUNCT',
  'left_end_word3': '-',
  'left_end_word2': '-',
  'right_end_word3': ':',
  'right_end_word2': ':'},
 {'word': ':',
  'left_neighbor': 'Zaman',
  'right_neighbor': 'American',
  'pos_in_sentence': 0.10714285714285714,
  'end_word3': ':',
  'end_word2': ':',
  'left_pos': 'PROPN',
  'left_

In [87]:
v = DictVectorizer(sparse=False)

In [88]:
X_matrix = v.fit_transform(X[:10000])

In [89]:
X_matrix.shape

(10000, 11889)

In [90]:
Y=np.array([train.pos[train.y[i]] for i in range(train.nb_tokens)])

In [91]:
Y

array(['PROPN', 'PUNCT', 'PROPN', ..., 'NOUN', 'PUNCT', 'PUNCT'],
      dtype='<U5')

In [92]:
pca = PCA(n_components=1000)
X_matrix=pca.fit_transform(X_matrix)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix[:10000], Y[:10000], test_size=0.2, random_state=42)

In [94]:
X_train.shape

(8000, 1000)

In [95]:
from sklearn.linear_model import Perceptron
clf = Perceptron(random_state=0)
clf.fit(X_train, y_train)

Perceptron()

In [96]:
clf.predict(X_test)

array(['VERB', 'ADJ', 'PRON', ..., 'NOUN', 'SYM', 'VERB'], dtype='<U5')

In [97]:
clf.score(X_test, y_test)

0.837

In [98]:
#3. position dans la phrase (ratio ou pos) (nb de mots dans la phrase)
#3. à côté ponctuation ou pas

#4. terminaison du mot (début)
#5. terminaison du mot précédent/suivant
# pos tag of left(/right) neighbors
